In [1]:
!pip install -v gptqmodel --no-build-isolation

Using pip 25.2 from /Users/atharvabot7/Downloads/MLOPS-Tutorials/Fine-Tuning-Practice/gptq/lib/python3.11/site-packages/pip (python 3.11)
  Preparing metadata (pyproject.toml) ...   Running command Preparing metadata (pyproject.toml)
  CUDA None
  HAS_CUDA_V8 False
  SETUP_KWARGS {}
  running dist_info
  creating /private/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/pip-modern-metadata-sbozxx1r/gptqmodel.egg-info
  writing /private/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/pip-modern-metadata-sbozxx1r/gptqmodel.egg-info/PKG-INFO
  writing dependency_links to /private/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/pip-modern-metadata-sbozxx1r/gptqmodel.egg-info/dependency_links.txt
  writing requirements to /private/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/pip-modern-metadata-sbozxx1r/gptqmodel.egg-info/requires.txt
  writing top-level names to /private/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/pip-modern-metadata-sbozxx1r/gptqmodel.egg-info/top_level.txt
  writin

In [3]:
!pip install "protobuf>=6.32.0"

  Using cached protobuf-6.33.0-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
Using cached protobuf-6.33.0-cp39-abi3-macosx_10_9_universal2.whl (427 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5


In [ ]:
import gptqmodel
dir(gptqmodel)

/Users/atharvabot7/Downloads/MLOPS-Tutorials/Fine-Tuning-Practice/gptq/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



WARN  Python GIL is enabled: Multi-gpu quant acceleration for MoE models is sub-optimal and multi-core accelerated cpu packing is also disabled. We recommend Python >= 3.13.3t with Pytorch > 2.8 for mult-gpu quantization and multi-cpu packing with env `PYTHON_GIL=0`.
WARN  Feature `utils/Perplexity` requires python GIL or Python >= 3.13.3T (T for Threading-Free edition of Python) plus Torch 2.8. Feature is currently skipped/disabled.
INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          
